In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def get_top_systems(drug_name, nest_map):
    rlipp_map = {}
    p_map = {}
    c_map = {}
    for i, row in rlipp_df.iterrows():
        rlipp_map[row['Term']] = (row['RLIPP'])
        p_map[row['Term']] = (row['P_rho'])
        c_map[row['Term']] = (row['C_rho'])
    p_map = {term:p_rho for term,p_rho in sorted(p_map.items(), key=lambda item:item[1], reverse=True)}

    subsys_df = pd.DataFrame()
    for i, t in enumerate(p_map):
        subsys_df = subsys_df.append({'Rank':(i+1), 'Term':t, 'Name':nest_map[t], 'P_rho':p_map[t], 'C_rho':c_map[t], 'RLIPP':rlipp_map[t]}, ignore_index = True)
        
    return subsys_df

In [ ]:
nest_df = pd.read_csv('../data/NeST/NeST_node.csv', sep=',')
nest_map = {row['name'].replace('.', '-'):row['Annotation'] for i, row in nest_df.iterrows()}

In [ ]:
ont = 'ctg'
dataset = 'av'
zscore_method = 'auc'
folds = 5

drugs = list(pd.read_csv('../data/training_files_av/drugname_av.txt', header=None, names=['D'])['D'])
drugs = ['Palbociclib']

for drug in drugs:
    for i in range(1, folds+1):
        modeldir = '../models/model_' + ont + '_' + dataset + '_' + drug + '_' + zscore_method + '_' + str(i)
        rlipp_df = pd.read_csv(modeldir + '/rlipp_genie_428.out', sep='\t')
        subsys_df = get_top_systems(drug, nest_map)
        subsys_df.to_csv(modeldir + '/subsystem_ranks_genie_428.txt', sep='\t', float_format='%.4f', index=False)

In [ ]:
for drug in drugs:
    agg_terms = []
    for i in range(1, folds+1):
        modeldir = '../models/model_' + ont + '_' + dataset + '_' + drug + '_' + zscore_method + '_' + str(i)
        subsys_df = pd.read_csv(modeldir + '/subsystem_ranks_genie_428.txt', sep='\t')[['Term', 'Name', 'P_rho', 'C_rho', 'RLIPP']]
        agg_terms.append(subsys_df)
    
    agg_df = pd.concat(agg_terms, ignore_index=True)
    agg_rlipp_df = pd.DataFrame(agg_df.groupby(['Term', 'Name']).mean()).reset_index()
    agg_rlipp_df = agg_rlipp_df.sort_values(by='P_rho', ascending=False)
    agg_rlipp_df.to_csv('../models/rlipp/' + drug + '_genie_428.txt', sep='\t', float_format='%.4f', index=False)